In [1]:
import keras 
import numpy as np 
path = keras.utils.get_file( 
'nietzsche.txt', 
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt') 
text = open(path).read().lower() 
print('Corpus length:', len(text))

600901/600901 ━━━━━━━━━━━━━━━━━━━━ 2s 4us/step
Corpus length: 600901


In [12]:
text[2:62]

'eface\n\n\nsupposing that truth is a woman--what then? is there'

In [16]:
maxlen = 60 
step = 3 
sentences = [] 
next_chars = [] 
for i in range(0, len(text) - maxlen, step): 
    sentences.append(text[i: i + maxlen]) 
    next_chars.append(text[i + maxlen]) 
print('Number of sequences:', len(sentences)) 
chars = sorted(list(set(text))) 
print('Unique characters:', len(chars)) 
char_indices = dict((char, chars.index(char)) for char in chars) 
print('Vectorization...') 
x = np.zeros((len(sentences), maxlen, len(chars)), dtype='bool') 
y = np.zeros((len(sentences), len(chars)), dtype='bool') 
for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        x[i, t, char_indices[char]] = 1 
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
Unique characters: 59
Vectorization...


In [28]:
x.shape

(200281, 60, 59)

In [29]:
y.shape

(200281, 59)

In [24]:
from keras import layers 
model = keras.models.Sequential() 
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars)))) 
model.add(layers.Dense(len(chars), activation='softmax'))

C:\Users\Mega Store\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01) 
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [27]:
def sample(preds, temperature=1.0): 
    preds = np.asarray(preds).astype('float64') 
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds) 
    preds = exp_preds / np.sum(exp_preds) 
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)

In [30]:
import random 
import sys 
for epoch in range(1, 60): 
    print('epoch', epoch) 
    model.fit(x, y, batch_size=128, epochs=1) 
    start_index = random.randint(0, len(text) - maxlen - 1) 
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"') 
    for temperature in [0.2, 0.5, 1.0, 1.2]: 
        print('------ temperature:', temperature) 
        sys.stdout.write(generated_text)
        for i in range(400): 
            sampled = np.zeros((1, maxlen, len(chars))) 
            for t, char in enumerate(generated_text): 
                sampled[0, t, char_indices[char]] = 1. 
            preds = model.predict(sampled, verbose=0)[0] 
            next_index = sample(preds, temperature) 
            next_char = chars[next_index] 
            generated_text += next_char 
            generated_text = generated_text[1:] 
            sys.stdout.write(next_char)

epoch 1
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 405s 255ms/step - loss: 2.3342
--- Generating with seed: "
that we fully appreciate what preparations, shifts, trials,"
------ temperature: 0.2

that we fully appreciate what preparations, shifts, trials, the strule and the sust and the stand the still the antanter and the stand to the stand with the extrule and the entans and the stand the expecient to the something the stand the entain to the stand the strung and the stand the stand the still as the some and the sime and the some and the sciention of the sangering the strung the experience the sense and the man the presention to the conserition ------ temperature: 0.5
nce the sense and the man the presention to the conserition to be the presient in the sension himself interiented in the flacitions and commanion as the will petuic and reales a may of stand be the manter the our all the manthere as its its was the sungerents to the themen the presure the the eate the say its such the signle and th

KeyboardInterrupt: 